# 🤟 VideoMAE WLASL Training - Sistema Completo

**Autor:** Rafael Ovalle - Tesis UNAB  
**Dataset:** WLASL100/WLASL300 (V1 y V2)  
**Modelo:** VideoMAE (MCG-NJU/videomae-base-finetuned-kinetics)  

---

## 📋 Estructura del Notebook

1. ✅ **Configuración Inicial** - Verificar GPU y entorno
2. 📦 **Clonar Repositorio y Dataset** - Desde GitHub y Google Drive
3. 🔍 **Verificar Dataset** - Estructura y splits
4. ⚙️ **Configurar Hiperparámetros** - Elegir dataset y versión
5. 🚀 **Entrenamiento** - Largo o corto de prueba
6. 📊 **Evaluación** - Listar modelos y evaluar
7. 📈 **Visualización** - Métricas y gráficos
8. 💾 **Descargar Resultados** - Comprimir y exportar
9. 🛠️ **Utilidades** - Monitorear GPU y limpiar memoria

---

**IMPORTANTE:**  
- Este notebook está diseñado para **Google Colab conectado a VM local**
- Usa `!python3` para comandos en la VM
- Dataset clonado con `shutil` desde Google Drive
- Ruta dataset: `/home/ov4lle/AtiendeSenas-MVP/data/`


---
# 1️⃣ Configuración Inicial


## 1.1 Verificar GPU


In [ ]:
# Verificar GPU con nvidia-smi
!nvidia-smi

Sun Nov 30 20:54:43 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.195.03             Driver Version: 570.195.03     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      On  |   00000000:00:03.0 Off |                    0 |
| N/A   65C    P8             18W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# Verificar GPU en PyTorch
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ GPU no disponible - entrenamiento será en CPU (muy lento)")

PyTorch version: 2.7.1+cu128
CUDA available: True
CUDA version: 12.8
GPU: NVIDIA L4
GPU Memory: 23.67 GB


## 1.2 Verificar Uso de Disco


In [ ]:
# Verificar espacio en disco
!df -h

Filesystem       Size  Used Avail Use% Mounted on
/dev/root        146G   43G  103G  30% /
tmpfs             16G     0   16G   0% /dev/shm
tmpfs            6.3G  996K  6.3G   1% /run
tmpfs            5.0M     0  5.0M   0% /run/lock
efivarfs          56K   24K   27K  48% /sys/firmware/efi/efivars
/dev/nvme0n1p15  105M  6.1M   99M   6% /boot/efi
tmpfs            3.2G  4.0K  3.2G   1% /run/user/1023


---
# 2️⃣ Clonar Repositorio y Dataset


**OPCIONAL** BORRAR TODOS LOS ARCHIVOS DE LA VM

In [ ]:
import shutil
import os

repo_path = "/home/ov4lle/AtiendeSenas-MVP"

if os.path.exists(repo_path):
    print(f"🗑️ Eliminando TODA la carpeta: {repo_path} ...")
    shutil.rmtree(repo_path)
    print("✅ Carpeta eliminada completamente.")
else:
    print(f"❌ La carpeta no existe: {repo_path}")


🗑️ Eliminando TODA la carpeta: /home/ov4lle/AtiendeSenas-MVP ...
✅ Carpeta eliminada completamente.


## 2.1 Clonar Repositorio desde GitHub


In [ ]:
import os
os.chdir("/home/ov4lle/")
# Configurar Git (opcional, para commits)
!git config --global user.email "ov4lle23@gmial.com"
!git config --global user.name "ov4llezz"

# Clonar repositorio si no existe
if not os.path.exists('AtiendeSenas-MVP'):
    print("[INFO] Clonando repositorio...")
    !git clone https://github.com/Ov4llezz/AtiendeSenas-MVP.git
    %cd AtiendeSenas-MVP
else:
    print("[INFO] Repositorio ya existe, actualizando...")
    %cd AtiendeSenas-MVP
    !git pull

print("\n✅ Repositorio listo")

[INFO] Clonando repositorio...
Cloning into 'AtiendeSenas-MVP'...
remote: Enumerating objects: 221, done.
remote: Counting objects: 100% (221/221), done.
remote: Compressing objects: 100% (191/191), done.
remote: Total 221 (delta 74), reused 172 (delta 28), pack-reused 0 (from 0)
Receiving objects: 100% (221/221), 4.77 MiB | 6.62 MiB/s, done.
Resolving deltas: 100% (74/74), done.
/home/ov4lle/AtiendeSenas-MVP

✅ Repositorio listo


/home/ov4lle/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Remover carpeta "data", para luego copiarl del drive completa


In [ ]:
import shutil
repo_path = "/home/ov4lle/AtiendeSenas-MVP/data"

if os.path.exists(repo_path):
    print(f"🗑️ Eliminando TODA la carpeta: {repo_path} ...")
    shutil.rmtree(repo_path)
    print("✅ Carpeta eliminada completamente.")
else:
    print(f"❌ La carpeta no existe: {repo_path}")

🗑️ Eliminando TODA la carpeta: /home/ov4lle/AtiendeSenas-MVP/data ...
✅ Carpeta eliminada completamente.


## 2.2 Clonar Dataset desde Google Drive


In [ ]:
import os
import shutil

# 1. INSTALAR UNZIP (Esto es lo que faltaba)
print("🔧 Instalando herramienta unzip...")
!sudo apt-get update > /dev/null
!sudo apt-get install -y unzip > /dev/null


# 2. Configurar rutas
destination_folder = "/home/ov4lle/AtiendeSenas-MVP"
zip_filename = "dataset_master.zip"
os.makedirs(destination_folder, exist_ok=True)

# 3. ID del archivo (El mismo de antes)
file_id = '1rjRGrepmD1mGtWS-QbN_EC8QIcUTHzW8'

# 4. Descargar de nuevo (porque el anterior se borró)
print(f"⬇️ Descargando de nuevo...")
url = f'https://drive.google.com/uc?id={file_id}'
output_path = f"{destination_folder}/{zip_filename}"

# Usamos gdown
!pip install -q gdown
!gdown {url} -O "{output_path}"

# 5. Descomprimir (Ahora sí funcionará)
print(f"📦 Descomprimiendo...")
if os.path.exists(output_path):
    !unzip -q -o "{output_path}" -d "{destination_folder}"

    # 6. Borrar el zip
    !rm "{output_path}"
    print("✅ ¡Listo! Dataset descomprimido correctamente.")
else:
    print("❌ Error: El archivo zip no se descargó.")

# Verificar archivos
!ls -lh "{destination_folder}" | head -n 10

🔧 Instalando herramienta unzip...
W: http://packages.cloud.google.com/apt/dists/apt-transport-artifact-registry-stable/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
W: https://us-apt.pkg.dev/projects/gce-ai-infra/dists/gpudirect-gib-apt/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
⬇️ Descargando de nuevo...
Downloading...
From (original): https://drive.google.com/uc?id=1rjRGrepmD1mGtWS-QbN_EC8QIcUTHzW8
From (redirected): https://drive.google.com/uc?id=1rjRGrepmD1mGtWS-QbN_EC8QIcUTHzW8&confirm=t&uuid=072953ea-d9bc-4f24-92de-212d566a1c69
To: /home/ov4lle/AtiendeSenas-MVP/dataset_master.zip
100%|██████████████████████████████████████| 9.11G/9.11G [01:33<00:00, 97.3MB/s]
📦 Descomprimiendo...
✅ ¡Listo! Dataset descomprimido correctamente.
total 9.8M
-rw-rw-r-- 1 ov4lle ov4lle  26K Nov 30 19:32 AtiendeSenas_Training_C

### Instalar Dependencias

In [ ]:
# Instalar todas las dependencias necesarias
%cd /home/ov4lle/AtiendeSenas-MVP
from colab_utils.config import setup_environment
print("[INFO] Instalando dependencias...\n")
setup_environment()
print("\n✅ Todas las dependencias instaladas")

/home/ov4lle/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/home/ov4lle/AtiendeSenas-MVP


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ov4lle/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/ov4lle/.local/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


[INFO] Instalando dependencias...

🔧 Installing dependencies...
✅ Dependencies installed


✅ Todas las dependencias instaladas


---
# 3️⃣ Verificar Dataset


In [1]:
# Cambiar al directorio del proyecto
%cd /home/ov4lle/AtiendeSenas-MVP

/home/ov4lle/AtiendeSenas-MVP


/home/ov4lle/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
# Verificar estructura de los 4 datasets
from colab_utils.dataset_utils import verify_all_datasets

verify_all_datasets(data_root="/home/ov4lle/AtiendeSenas-MVP/data")

                            VERIFICACION DE DATASETS                            
Ruta base: /home/ov4lle/AtiendeSenas-MVP/data


--------------------------------------------------------------------------------
[Dataset] WLASL100 - V1 Original
--------------------------------------------------------------------------------
[OK] Dataset encontrado
Ruta: /home/ov4lle/AtiendeSenas-MVP/data/wlasl100

Splits:
  - train     :  807 videos
  - val       :  194 videos
  - test      :  117 videos
  ----------------------------------------
  - TOTAL     : 1118 videos
  - En disco  : 1232 videos

Notas:
  [INFO] Hay 114 videos extras en disco (Total: 1232, En splits: 1118)

--------------------------------------------------------------------------------
[Dataset] WLASL300 - V1 Original
--------------------------------------------------------------------------------
[OK] Dataset encontrado
Ruta: /home/ov4lle/AtiendeSenas-MVP/data/wlasl300

Splits:
  - train     : 1960 videos
  - val       :  558 vid

---
# 4️⃣ Configurar Hiperparámetros y Elegir Dataset


## 4.1 Configuración de Hiperparámetros

**Parámetros disponibles:**
- `dataset_type`: "wlasl100" o "wlasl300"
- `version`: "v1" (train/val/test separados) o "v2" (train+val combinados)
- `batch_size`: Tamaño del batch (default: 6)
- `max_epochs`: Número máximo de epochs (default: 30)
- `learning_rate`: Learning rate inicial (default: 1e-5)
- `patience`: Epochs sin mejora para early stopping (default: 10)
- `weight_decay`: Regularización L2 (default: 0.0)
- `label_smoothing`: Label smoothing (default: 0.0)
- `class_weighted`: Usar pesos de clases (default: False)
- `freeze_backbone`: Congelar backbone VideoMAE (default: False)


In [2]:
from colab_utils.config import create_config, print_config

# ============================================================
#   CONFIGURACIÓN DEL EXPERIMENTO
# ============================================================

config = create_config(
    # Dataset y versión
    dataset_type="wlasl100",      # "wlasl100" o "wlasl300"
    version="v2",                  # "v1" o "v2"
    data_root="/home/ov4lle/AtiendeSenas-MVP/data",

    # Hiperparámetros principales
    batch_size=6,
    max_epochs=45,
    learning_rate=8e-5,
    patience=12,

    # Regularización
    weight_decay=0.04,
    label_smoothing=0.07,
    class_weighted=True,
    freeze_backbone=False,         # False = entrenar todas las capas

    # Otros
    warmup_ratio=0.07,
    min_lr=1e-5,
    gradient_clip=1.0,
    num_workers=6,
    save_every=5
)

# Mostrar configuración
print_config(config)

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ov4lle/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/ov4lle/.local/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


\n======================================================================
                    CONFIGURACIÓN DEL EXPERIMENTO                     
Dataset: WLASL100 (100 clases)
Versión: V2
Dataset Name: wlasl100_v2
\n─────────────────────Hiperparámetros Principales──────────────────────
  Batch Size:         6
  Max Epochs:         45
  Learning Rate:      8.00e-05
  Patience (E.Stop):  12 epochs
\n────────────────────────────Regularización────────────────────────────
  Weight Decay:       0.04
  Label Smoothing:    0.07
  Class Weighted:     True
\n────────────────────────────────Modelo────────────────────────────────
  Freeze Backbone:    False
  Gradient Clip:      1.0
\n──────────────────────────────Scheduler───────────────────────────────
  Warmup Ratio:       0.07
  Min LR:             1.00e-05
\n────────────────────────────────Rutas─────────────────────────────────
  Data:        /home/ov4lle/AtiendeSenas-MVP/data/wlasl100_v2
  Checkpoints: /home/ov4lle/AtiendeSenas-MVP/data/../mo

---
# 5️⃣ Entrenamiento


In [ ]:
# Recargar módulos para que tome los
import sys
import importlib

# Eliminar módulo del cache si ya fue importado
if 'colab_utils.dataset' in sys.modules:
  del sys.modules['colab_utils.dataset']

# Ahora importar nuevamente
from colab_utils.dataset import create_dataloaders

print("✅ Módulo recargado correctamente")

✅ Módulo recargado correctamente


## 5.1 Entrenamiento CORTO (Prueba - 3 epochs)


In [ ]:
# Entrenamiento corto de prueba (3 epochs)
from colab_utils.training import train_model
from colab_utils.dataset import create_dataloaders

# Crear dataloaders
train_loader, val_loader, train_dataset = create_dataloaders(config)

# Configuración temporal para prueba rápida
config_prueba = config.copy()
config_prueba['max_epochs'] = 1
config_prueba['save_every'] = 1

print("\n🚀 Iniciando entrenamiento de PRUEBA (3 epochs)...\n")

# Entrenar
model, history, checkpoint_dir, log_dir = train_model(
    config_prueba,
    train_loader,
    val_loader,
    train_dataset
)

print(f"\n✅ Entrenamiento de prueba completado")
print(f"📁 Checkpoints: {checkpoint_dir}")
print(f"📊 Logs: {log_dir}")

[INFO] Creando dataloaders para: wlasl100_v2
[INFO] Base path: /home/ov4lle/AtiendeSenas-MVP/data/wlasl100_v2
[INFO] Train dataset: 1001 muestras
[INFO] Val dataset: 117 muestras
[INFO] Train batches: 251
[INFO] Val batches: 30

🚀 Iniciando entrenamiento de PRUEBA (3 epochs)...

\n======================================================================
                     INICIALIZANDO ENTRENAMIENTO                      
======================================================================\n


/home/ov4lle/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of VideoMAEForVideoClassification were not initialized from the model checkpoint at MCG-NJU/videomae-base-finetuned-kinetics and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([400]) in the checkpoint and torch.Size([100]) in the model instantiated
- classifier.weight: found shape torch.Size([400, 768]) in the checkpoint and torch.Size([100, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ TODAS las 12 capas del modelo VideoMAE son entrenables
Modelo: MCG-NJU/videomae-base-finetuned-kinetics
Parámetros totales: 86,304,100
Parámetros entrenables: 86,304,100
Parámetros congelados: 0\n
Optimizer: AdamW (lr=1.00e-05, wd=0.0)
Scheduler: Warmup + Cosine Decay
Total steps: 251, Warmup: 25\n
                       INICIO DEL ENTRENAMIENTO                       
======================================================================\n
\n======================================================================
EPOCH 1/1


\n======================================================================
RESULTADOS EPOCH 1
Train Loss: 9.7216 | Train Acc: 0.80%
Val Loss:   6.6844 | Val Acc:   1.67%
LR actual:  1.00e-06
======================================================================\n
[CHECKPOINT] Guardado: epoch_1.pt
[BEST MODEL] Val Loss: 6.6844 | Val Acc: 1.67%
\n======================================================================
                       ENTRENAMIENTO COMPLETADO                       
Mejor Val Loss: 6.6844
Mejor Val Accuracy: 1.67%
Checkpoints: /home/ov4lle/AtiendeSenas-MVP/data/../models/v2/wlasl100_v2/checkpoints/run_20251130_201058
Logs: /home/ov4lle/AtiendeSenas-MVP/data/../runs/v2/wlasl100_v2/run_20251130_201058
======================================================================\n

✅ Entrenamiento de prueba completado
📁 Checkpoints: /home/ov4lle/AtiendeSenas-MVP/data/../models/v2/wlasl100_v2/checkpoints/run_20251130_201058
📊 Logs: /home/ov4lle/AtiendeSenas-MVP/data/../runs/v2/wla

## 5.2 Entrenamiento COMPLETO (30 epochs)


In [3]:
# Entrenamiento completo (30 epochs o lo configurado)
from colab_utils.training import train_model
from colab_utils.dataset import create_dataloaders

# Crear dataloaders
train_loader, val_loader, train_dataset = create_dataloaders(config)

print(f"\n🚀 Iniciando entrenamiento COMPLETO ({config['max_epochs']} epochs)...\n")

# Entrenar
model, history, checkpoint_dir, log_dir = train_model(
    config,
    train_loader,
    val_loader,
    train_dataset
)

print(f"\n✅ Entrenamiento completado")
print(f"📁 Checkpoints: {checkpoint_dir}")
print(f"📊 Logs: {log_dir}")

[INFO] Creando dataloaders para: wlasl100_v2
[INFO] Base path: /home/ov4lle/AtiendeSenas-MVP/data/wlasl100_v2
[INFO] Train dataset: 1001 muestras
[INFO] Val dataset: 117 muestras
[INFO] Train batches: 167
[INFO] Val batches: 20

🚀 Iniciando entrenamiento COMPLETO (45 epochs)...

\n======================================================================
                     INICIALIZANDO ENTRENAMIENTO                      
======================================================================\n


/home/ov4lle/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of VideoMAEForVideoClassification were not initialized from the model checkpoint at MCG-NJU/videomae-base-finetuned-kinetics and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([400]) in the checkpoint and torch.Size([100]) in the model instantiated
- classifier.weight: found shape torch.Size([400, 768]) in the checkpoint and torch.Size([100, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[INFO] Reinicializando classifier head con std=0.01...
✅ Classifier head reinicializado (std=0.01)
✅ TODAS las 12 capas del modelo VideoMAE son entrenables
[INFO] Verificando inicialización del modelo...
✅ Modelo inicializado correctamente (sin NaN/Inf)
Modelo: MCG-NJU/videomae-base-finetuned-kinetics
Parámetros totales: 86,304,100
Parámetros entrenables: 86,304,100
Parámetros congelados: 0\n
[INFO] Calculando class weights...
Class weights (min=0.532, max=1.916)\n
Optimizer: AdamW (lr=8.00e-05, wd=0.04)
Scheduler: Warmup + Cosine Decay
Total steps: 7515, Warmup: 526\n
                       INICIO DEL ENTRENAMIENTO                       
======================================================================\n
\n======================================================================
EPOCH 1/45


\n======================================================================
RESULTADOS EPOCH 1
Train Loss: 4.6326 | Train Acc: 1.30%
Val Loss:   4.5788 | Val Acc:   2.50%
LR actual:  2.54e-05
======================================================================\n
[CHECKPOINT] Guardado: epoch_1.pt
[BEST MODEL] Val Loss: 4.5788 | Val Acc: 2.50%
\n======================================================================
EPOCH 2/45


\n======================================================================
RESULTADOS EPOCH 2
Train Loss: 4.4844 | Train Acc: 4.93%
Val Loss:   4.2982 | Val Acc:   5.83%
LR actual:  5.08e-05
======================================================================\n
[CHECKPOINT] Guardado: epoch_2.pt
[BEST MODEL] Val Loss: 4.2982 | Val Acc: 5.83%
\n======================================================================
EPOCH 3/45


...

\n======================================================================
RESULTADOS EPOCH 41
Train Loss: 0.6032 | Train Acc: 99.40%
Val Loss:   2.0234 | Val Acc:   63.33%
LR actual:  1.16e-05
======================================================================\n
\n======================================================================
EPOCH 42/45


\n======================================================================
RESULTADOS EPOCH 42
Train Loss: 0.6019 | Train Acc: 99.30%
Val Loss:   2.0310 | Val Acc:   61.67%
LR actual:  1.09e-05
======================================================================\n
\n======================================================================
EPOCH 43/45


\n======================================================================
RESULTADOS EPOCH 43
Train Loss: 0.6033 | Train Acc: 99.40%
Val Loss:   2.0477 | Val Acc:   61.67%
LR actual:  1.04e-05
======================================================================\n
\n======================================================================
EPOCH 44/45


\n======================================================================
RESULTADOS EPOCH 44
Train Loss: 0.6035 | Train Acc: 99.40%
Val Loss:   2.0401 | Val Acc:   62.50%
LR actual:  1.01e-05
======================================================================\n
\n======================================================================
EPOCH 45/45


\n======================================================================
RESULTADOS EPOCH 45
Train Loss: 0.6043 | Train Acc: 99.18%
Val Loss:   2.0154 | Val Acc:   61.67%
LR actual:  1.00e-05
======================================================================\n
[CHECKPOINT] Guardado: epoch_45.pt
\n======================================================================
                       ENTRENAMIENTO COMPLETADO                       
Mejor Val Loss: 1.9813
Mejor Val Accuracy: 64.17%
Checkpoints: /home/ov4lle/AtiendeSenas-MVP/data/../models/v2/wlasl100_v2/checkpoints/run_20251201_192405
Logs: /home/ov4lle/AtiendeSenas-MVP/data/../runs/v2/wlasl100_v2/run_20251201_192405
======================================================================\n

✅ Entrenamiento completado
📁 Checkpoints: /home/ov4lle/AtiendeSenas-MVP/data/../models/v2/wlasl100_v2/checkpoints/run_20251201_192405
📊 Logs: /home/ov4lle/AtiendeSenas-MVP/data/../runs/v2/wlasl100_v2/run_20251201_192405


In [4]:
!ls -l /home/ov4lle/AtiendeSenas-MVP/models/v2/wlasl100_v2/checkpoints/run_20251201_192405


total 24278888
-rw-rw-r-- 1 ov4lle ov4lle 1035887097 Dec  2 00:51 best_model.pt
-rw-rw-r-- 1 ov4lle ov4lle 1035892969 Dec  1 19:33 checkpoint_epoch_1.pt
-rw-rw-r-- 1 ov4lle ov4lle 1035893911 Dec  1 20:55 checkpoint_epoch_10.pt
-rw-rw-r-- 1 ov4lle ov4lle 1035893911 Dec  1 21:04 checkpoint_epoch_11.pt
-rw-rw-r-- 1 ov4lle ov4lle 1035893911 Dec  1 21:40 checkpoint_epoch_15.pt
-rw-rw-r-- 1 ov4lle ov4lle 1035892969 Dec  1 19:42 checkpoint_epoch_2.pt
-rw-rw-r-- 1 ov4lle ov4lle 1035893911 Dec  1 22:26 checkpoint_epoch_20.pt
-rw-rw-r-- 1 ov4lle ov4lle 1035893911 Dec  1 22:35 checkpoint_epoch_21.pt
-rw-rw-r-- 1 ov4lle ov4lle 1035893911 Dec  1 22:53 checkpoint_epoch_23.pt
-rw-rw-r-- 1 ov4lle ov4lle 1035893911 Dec  1 23:11 checkpoint_epoch_25.pt
-rw-rw-r-- 1 ov4lle ov4lle 1035893911 Dec  1 23:39 checkpoint_epoch_28.pt
-rw-rw-r-- 1 ov4lle ov4lle 1035892969 Dec  1 19:51 checkpoint_epoch_3.pt
-rw-rw-r-- 1 ov4lle ov4lle 1035893911 Dec  1 23:57 checkpoint_epoch_30.pt
-rw-rw-r-- 1 ov4lle ov4lle 10358939

In [6]:
import os
import shutil

# Carpeta origen (checkpoint)
src = "/home/ov4lle/AtiendeSenas-MVP/models/v2/wlasl100_v2/checkpoints/run_20251201_192405"

# Carpeta destino (dentro de descargas/V1/wlasl100/run_20251201_124141)
dst = "/home/ov4lle/AtiendeSenas-MVP/descargas/V2/wlasl100/run_20251201_192405"


# Crear carpeta destino si no existe
os.makedirs(dst, exist_ok=True)

# Archivos que queremos copiar
files_to_copy = [
    "best_model.pt",
    "config.json",
    "training_history.csv"
]

# Copiar uno por uno manteniendo el mismo nombre
for fname in files_to_copy:
    shutil.copy(os.path.join(src, fname),
                os.path.join(dst, fname))

# (Opcional) Verificamos qué quedó en la carpeta destino
!ls -lh "/home/ov4lle/AtiendeSenas-MVP/descargas/V2/wlasl100/run_20251201_192405"


total 988M
-rw-rw-r-- 1 ov4lle ov4lle 988M Dec  2 02:51 best_model.pt
-rw-rw-r-- 1 ov4lle ov4lle  785 Dec  2 02:51 config.json
-rw-rw-r-- 1 ov4lle ov4lle 4.3K Dec  2 02:51 training_history.csv


---
# 6️⃣ Evaluación de Modelo


## 6.1 Listar Modelos Disponibles


In [ ]:
# Listar modelos V1 (wlasl100/300 originales)
!python3 scripts/test.py --list

/home/ov4lle/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/ov4lle/.local/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
[INFO] Backend de Matplotlib cambiado a 'Agg' (no interactivo)
[ERROR] Directorio models/checkpoints no existe
[INFO] No hay runs disponibles


In [ ]:
# Listar modelos V2 (wlasl100_v2/300_v2 - train+val combinados)
!python3 scripts_v2/test.py --list

/home/ov4lle/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/ov4lle/.local/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
[INFO] Backend de Matplotlib cambiado a 'Agg' (no interactivo)
[WARN] No se pudieron leer métricas de run_20251130_201058: Weights only load failed. This file can still be loaded, to do so you have two options, do those steps only if you trust the source of the checkpoint. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but 

## 6.2 Evaluar Modelo por ID


In [ ]:
# Evaluar modelo V1 por ID
# Ejemplo: si el ID es 1, evalúa el run #1

MODEL_ID_V1 = 1  # CAMBIAR según el listado

!python3 scripts/test.py --run-id {MODEL_ID_V1}

/home/ov4lle/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/ov4lle/.local/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
[INFO] Backend de Matplotlib cambiado a 'Agg' (no interactivo)
[ERROR] Directorio models/checkpoints no existe
[ERROR] No hay runs disponibles

Usa --list-runs para ver los runs disponibles


In [ ]:
# Evaluar modelo V2 por ID
MODEL_ID_V2 = 1  # CAMBIAR según el listado

!python3 scripts_v2/test.py --run-id {MODEL_ID_V2}

/home/ov4lle/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/ov4lle/.local/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
[INFO] Backend de Matplotlib cambiado a 'Agg' (no interactivo)

[INFO] Run seleccionado: run_20251130_201058
[INFO] Epoch: 1
[INFO] Val Accuracy: 1.6666666666666667
[INFO] Val Loss: 6.684448766708374


                   CARGANDO MODELO Y CONFIGURACIÓN                    

[INFO] Cargando checkpoint: models/v2/wlasl100_v2/checkpoints/run_20251130_201058/best_model.pt
[INFO] Modelo: MCG-NJU/videomae-base-finetuned-kinetics
[INFO] Num classes: 100
/home/ov4lle/.local/lib/python

## 6.3 Evaluar Modelo por Path Específico


In [ ]:
# Evaluar modelo V1 por path específico
CHECKPOINT_PATH_V1 = "/home/ov4lle/AtiendeSenas-MVP/models/v2/wlasl100_v2/checkpoints/run_20251130_201058"  # CAMBIAR

!python3 scripts/test.py --checkpoint {CHECKPOINT_PATH_V1}

/home/ov4lle/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/ov4lle/.local/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
[INFO] Backend de Matplotlib cambiado a 'Agg' (no interactivo)
usage: test.py [-h] [--checkpoint_path CHECKPOINT_PATH] [--run-id RUN_ID]
               [--list-runs] [--checkpoint_dir CHECKPOINT_DIR]
               [--base_path BASE_PATH] [--batch_size BATCH_SIZE]
               [--num_workers NUM_WORKERS] [--output_dir OUTPUT_DIR]
               [--device DEVICE]
test.py: error: ambiguous option: --checkpoint could match --checkpoint_path, --checkpoint_dir


In [ ]:
# Evaluar modelo V2 por path específico
CHECKPOINT_PATH_V2 = "/home/ov4lle/AtiendeSenas-MVP/models/v2/wlasl100_v2/checkpoints/run_20251130_201058t"  # CAMBIAR

!python3 scripts_v2/test.py --checkpoint {CHECKPOINT_PATH_V2}

/home/ov4lle/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/ov4lle/.local/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
[INFO] Backend de Matplotlib cambiado a 'Agg' (no interactivo)
usage: test.py [-h] [--checkpoint_path CHECKPOINT_PATH] [--run-id RUN_ID]
               [--list-runs] [--checkpoint_dir CHECKPOINT_DIR]
               [--base_path BASE_PATH] [--batch_size BATCH_SIZE]
               [--num_workers NUM_WORKERS] [--output_dir OUTPUT_DIR]
               [--device DEVICE]
test.py: error: ambiguous option: --checkpoint could match --checkpoint_path, --checkpoint_dir


---
# 7️⃣ Visualización de Resultados


## 7.1 Matriz de Confusión


In [ ]:
from IPython.display import Image, display
import glob

# Buscar la matriz de confusión más reciente
confusion_matrices = sorted(glob.glob("evaluation_results*/confusion_matrix.png"), reverse=True)

if confusion_matrices:
    print(f"📊 Mostrando: {confusion_matrices[0]}")
    display(Image(filename=confusion_matrices[0]))
else:
    print("⚠️ No se encontraron matrices de confusión. Ejecuta la evaluación primero.")

## 7.2 Métricas Detalladas


In [ ]:
import json

# Buscar el reporte JSON más reciente
json_reports = sorted(glob.glob("evaluation_results*/test_results_*.json"), reverse=True)

if json_reports:
    with open(json_reports[0], 'r') as f:
        results = json.load(f)

    print("="*70)
    print(f"{'RESULTADOS DE EVALUACIÓN':^70}")
    print("="*70)
    print(f"Accuracy Total: {results['overall_accuracy']:.2f}%")
    print(f"Top-3 Accuracy: {results.get('top3_accuracy', 'N/A')}")
    print(f"Top-5 Accuracy: {results.get('top5_accuracy', 'N/A')}")
    print(f"\nPrecision (macro): {results.get('precision_macro', 'N/A')}")
    print(f"Recall (macro): {results.get('recall_macro', 'N/A')}")
    print(f"F1-score (macro): {results.get('f1_macro', 'N/A')}")
    print("="*70)
else:
    print("⚠️ No se encontraron reportes JSON. Ejecuta la evaluación primero.")

## 7.3 Curvas de Entrenamiento (TensorBoard)


In [ ]:
# Cargar TensorBoard
%load_ext tensorboard

# Mostrar logs del último run
# CAMBIAR según el directorio del último run
LOG_DIR = "runs/v2/wlasl100_v2"  # o "runs_v2/..."

%tensorboard --logdir {LOG_DIR}

---
# 8️⃣ Descargar Resultados


## 8.1 Comprimir y Descargar Todo


In [ ]:
import shutil
from datetime import datetime

# Crear archivo zip con todos los resultados
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_zip = f"resultados_completos_{timestamp}"

# Agregar al zip: checkpoints, logs, evaluation_results
!zip -r {output_zip}.zip models/ models_v2/ runs/ runs_v2/ evaluation_results/ evaluation_results_v2/

print(f"\n✅ Archivo creado: {output_zip}.zip")
print(f"📦 Tamaño: ", end="")
!du -h {output_zip}.zip

In [ ]:
# Descargar archivo zip a tu máquina local
from google.colab import files

files.download(f"{output_zip}.zip")

## 8.2 Descargar Archivos Individuales


In [ ]:
# Descargar matriz de confusión
if confusion_matrices:
    files.download(confusion_matrices[0])

# Descargar reporte JSON
if json_reports:
    files.download(json_reports[0])

# Descargar mejor modelo
best_models = sorted(glob.glob("models*/checkpoints/*/best_model.pt"), reverse=True)
if best_models:
    files.download(best_models[0])

## 8.3 Copiar a Google Drive


In [ ]:
# Copiar resultados a Google Drive
DRIVE_OUTPUT_PATH = "/content/drive/MyDrive/TESIS/resultados"  # CAMBIAR ESTO

os.makedirs(DRIVE_OUTPUT_PATH, exist_ok=True)

# Copiar zip
shutil.copy(f"{output_zip}.zip", DRIVE_OUTPUT_PATH)

print(f"✅ Resultados copiados a: {DRIVE_OUTPUT_PATH}")

---
# 9️⃣ Utilidades


## 9.1 Monitorear GPU en Tiempo Real


In [ ]:
# Monitorear GPU cada 2 segundos (CTRL+C para detener)
!watch -n 2 nvidia-smi

## 9.2 Limpiar Memoria GPU


In [7]:
# Limpiar memoria GPU
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

print("✅ Memoria GPU limpiada")
print(f"Memoria GPU usada: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
print(f"Memoria GPU reservada: {torch.cuda.memory_reserved() / 1e9:.2f} GB")

✅ Memoria GPU limpiada
Memoria GPU usada: 0.71 GB
Memoria GPU reservada: 7.59 GB


## 9.3 Ver Procesos en GPU


In [ ]:
# Ver qué procesos están usando la GPU
!nvidia-smi --query-compute-apps=pid,process_name,used_memory --format=csv

---
# ✅ Notebook Completado

**Autor:** Rafael Ovalle - Tesis UNAB  
**Versión:** Final  
**Fecha:** Noviembre 2025  

---

## 📌 Notas Finales

- Este notebook está optimizado para **Google Colab + VM local**
- Usa `!python3` para todos los comandos
- Los datasets V2 ya tienen `train = train + val`
- Modelos V1 y V2 están separados en directorios distintos
- Recuerda ajustar las rutas de Google Drive según tu estructura

---

## 🚀 Próximos Pasos

1. Entrenar con configuraciones óptimas
2. Evaluar en test set
3. Analizar matriz de confusión y errores
4. Iterar con ajustes de hiperparámetros
5. Documentar resultados para la tesis

---

**¡Buena suerte con el entrenamiento! 🤟**
